# c3LTC

In this notebook, we implement the locally testible code with constant rate, distance and locallity proposed in [Dinur et. al.](https://arxiv.org/abs/2111.04808).


# Table of contents
1. [Representation of Squares and Edges](#representation)
2. [Tensor Decoding](#tensor)
3. [Parity check generation](#parity)
4. [Random generator sets](#sets)
5. [c3LTC object](#c3ltc)
6. [Concrete Example](#example)
7. [Encoding, Decoding and Testing](#tests)

In [1]:
from src.visualize_c3ltc import *
import numpy
import random
from pyvis import network as net

from sage.coding.grs_code import ReedSolomonCode
from sage.rings.finite_rings.finite_field_prime_modn import FiniteField_prime_modn as GF
from sage.matrix.matrix_space import MatrixSpace
from sage.coding.linear_code import LinearCode
from sage.modules.vector_modn_dense import vector
from sage.groups.matrix_gps.matrix_group import *

from src.row_reduce_from_c import row_reduce_and_orthogonal

# Representation of Squares and Edges  <a name="representation"></a>

Following are classes for describing squares and edges in left-right Cayley graph. Given a group  $G$ and sets $A,B$. 

$A$-edge is described as $(a,g)$ for $g\in G, a\in A$, and $B$-edge is described as $(g,b)$ for $g\in G, b
\in B$. The edge $(a,g)$ represent the edge between the vertices (i.e. group elements) $ag$ and $g$, and the edge $(g,b)$ represents the edge between $gb$ and $g$. Notice that an edge has several equivalent representation: $(a^{-1},ag)$, represents the same edge as $(a,g)$. Initializing ``AEdge(G,a,g)`` with either one of the representation will give an identical edge, e.g. ``AEdge(G,a,g) == AEdge(a^(-1),ag)``. The same holds for $B$-edges respectivley.  

A square by a tuple $(a,g,b)$ for $g\in G, a\in A, b\in B$. Similarly to the edges, a squre has several equivalent representations as well: $(a^{-1},ag,b)$, $(a^{-1},agb,b^{-1})$ and $(a,gb,b^{-1})$ all represent the same square. Initializing ``Square(G,a,g,b)`` with either one of the representation will give an identical square, e.g. ``Square(G,a,g,b) == Square(G, a^(-1),ag,b)``. 


In [2]:
class AEdge:
    def __init__(self, G, a, g):
        edge = self.canonical_a_edge(G, a, g)
        self.a = edge[0]
        self.g = edge[1]

    def canonical_a_edge(self, G, a, g):
        element_list = list(G)
        index_g = element_list.index(g)
        index_ag = element_list.index(a * g)
        min_index = min(index_g, index_ag)
        if min_index == index_g:
            return (a, g)
        if min_index == index_ag:
            return (a.inverse(), a * g)

    def __repr__(self):
        return "AEdge(%s, %s)" % (self.a, self.g)

    def __eq__(self, other):
        if isinstance(other, AEdge):
            return (self.a == other.a and self.g == other.g)
        else:
            return False

    def __ne__(self, other):
        return (not self.__eq__(other))

    def __hash__(self):
        return hash(self.__repr__())


class BEdge:
    def __init__(self, G, g, b):
        edge = self.canonical_B_edge(G, g, b)
        self.g = edge[0]
        self.b = edge[1]

    def canonical_B_edge(self, G, g, b):
        element_list = list(G)
        index_g = element_list.index(g)
        index_gb = element_list.index(g * b)
        min_index = min(index_g, index_gb)
        if min_index == index_g:
            return (g, b)
        if min_index == index_gb:
            return (g * b, b.inverse())

    def __repr__(self):
        return "BEdge(%s, %s)" % (self.g, self.b)

    def __eq__(self, other):
        if isinstance(other, BEdge):
            return (self.g == other.g and self.b == other.b)
        else:
            return False

    def __ne__(self, other):
        return (not self.__eq__(other))

    def __hash__(self):
        return hash(self.__repr__())


class Square:
    def __init__(self, G, a, g, b):
        square = self.canonical_square(G, a, g, b)
        self.a = square[0]
        self.g = square[1]
        self.b = square[2]

    def canonical_square(self, G, a, g, b):
        element_list = list(G)
        option1 = str((a, g, b))
        option2 = str((a.inverse(), a * g, b))
        option3 = str((a, g * b, b.inverse()))
        option4 = str((a.inverse(), a * g * b, b.inverse()))
        minimal = min(option1, option2, option3, option4)
        if minimal == option1:
            return (a, g, b)
        if minimal == option2:
            return (a.inverse(), a * g, b)
        if minimal == option3:
            return (a, g * b, b.inverse())
        if minimal == option4:
            return (a.inverse(), a * g * b, b.inverse())

    def __repr__(self):
        return "Square(%s, %s, %s)" % (self.a, self.g, self.b)

    def __eq__(self, other):
        if isinstance(other, Square):
            return (self.a == other.a and self.b == other.b and self.g == other.g)
        else:
            return False

    def __ne__(self, other):
        return (not self.__eq__(other))

    def __hash__(self):
        return hash(self.__repr__())


# Tensor Decoding <a name="tensor"></a>

Below we implement a simple tensor decoding algorithm. Given a word and two codes $C_A,C_B$, the decoding algorithm for $C_A\otimes C_B$ first decode all rows with $C_B$, then all columns with $C_A$. This algorithm corrects up to 
$(d_1d_2-1)/4$ errors. 

Here, we adopt a slight improvement of this algorithm, instead of decoding all the rows and then all the columns, we keep two sets called ``iterating_set_code_a`` and ``iterating_set_code_b``. Upon a correction of row $i$ (related to code $C_B$), each entry that was change signifies a column that needs to be checked (as it might not be in the code due to the change). Thus we'll add this column to ``iterating_set_code_b``. Iterating over ``iterating_set_code_b``, we do the same and update ``iterating_set_code_a``. 


In [3]:
def tensor_decoding(tensor_word, C_A: LinearCode, C_B: LinearCode):
    """ Returns a decoded word in tensor code. 

    Keyword arguments:
    tensor_word -- matrix of length(C_A) x length(C_B).
    C_A -- Sage code object.
    C_B -- Sage code object.
    """

    n_a = len(C_A.parity_check_matrix().columns())
    n_b = len(C_B.parity_check_matrix().columns())
    field = C_A.base_field()

    def tensor_word_to_tuple(m):
        return tuple(m.reshape((n_a * n_b)))

    corrected_word = numpy.copy(tensor_word)
    iterating_set_C_B = [i for i in range(n_a)]
    iterating_set_C_A = [i for i in range(n_b)]
    past_words = []
    word_to_tuple = tensor_word_to_tuple(corrected_word)
    while (len(iterating_set_C_A) != 0 or len(iterating_set_C_B) != 0) and word_to_tuple not in past_words:
        past_words.append(word_to_tuple)

        new_iterating_C_A = []
        new_iterating_C_B = []
        for i in iterating_set_C_B:
            local_word = vector(field, corrected_word[i, :])
            try:
                corrected_locally = C_B.decode_to_code(local_word)
            except:
                new_iterating_C_B.append(i)
                continue
            for j in range(n_b):
                if local_word[j] != corrected_locally[j]:
                    new_iterating_C_A.append(j)
                corrected_word[i][j] = corrected_locally[j]
        for j in iterating_set_C_A:
            local_word = vector(field, corrected_word[:, j])
            try:
                corrected_locally = C_A.decode_to_code(local_word)
            except:
                new_iterating_C_A.append(j)
                continue
            for i in range(n_a):
                if local_word[i] != corrected_locally[i]:
                    new_iterating_C_B.append(i)
                corrected_word[i][j] = corrected_locally[i]
        iterating_set_C_A = set(new_iterating_C_A)
        iterating_set_C_B = set(new_iterating_C_B)
        word_to_tuple = tensor_word_to_tuple(corrected_word)
    return corrected_word


# Generating the parity check matrix of the code <a name="parity"></a>

The following function reterns a representation of the parity check matrix of $c^3LTC$ code. It gets a group $G$, two sets $A,B$ and two codes $C_A,C_B$, such that $n_A = |A|, n_B = |B|$ and $C_A,C_B$ are defined on the same field. 

The function proceeds in two stages:
1. In the first stage, it collects the edges (by $A,B$) into sets.
2. In the seocnd stage, the local constraint on each edge are injected according to the squares.
    - Each edges "sees" a tuple of values squares such that the values on those squares 
      should be contained in either $C_A$ or $C_B$ (depending on whether it's an $A$-edge or $B$-edge).
    - To enforce the condition above, each one of these squares is associated with a column from the
      parity check of the local code ($C_A$ or $C_B$). The squares indicate the specific squares that
      participate in the constraints on that edge. 
    - The dictionary (mapping) ''constraints'' in the code, holds a sparse representation of the constraints
      in the parity check induced by the input parameters. It maps a square object (that indicates a column) 
      to a dictionary whose keys are row numbers and the values are the values in the large parity check matrix. 
    - In other words, a copy of the local parity check is being injected into the squares-parity-check
      such that each column of the small parity check is placed into the column associated with different square
      (according to the squares specified by the row). 
      
The function returns a sparse representation of the parity check. It returns a dictionary (mapping) with ``Square`` elements as keys. Each square is mapped to a sparse representation of the corresponding column to this square in the parity check matrix. Namley, it contains a mapping from row number to value. Hence non zero values in the parity check matrix are specified first by their ``Square``, then by their row, and finally by their value.  

In [4]:
def embedding_local_parity_constraints_on_squares(C_A, C_B, G, A, B):
    """ Returns
    1) Sparse representation of the constrsints by a mapping of squares (represeted by 3 group elements (a,g,b))
        to dictionary whose keys are rows in which the square has non zero value, and value is the value in the relevant row and column.
    2) Number constraints of rows in the constraint matrix.

    Keyword arguments:
    C_A -- Sage code object.
    C_B -- Sage code object.
    G -- Sage group object.
    A -- list of group elements.
    B -- list of group elements.

    """

    constraints = {}
    row = 0
    parity_A = C_A.parity_check_matrix()
    parity_B = C_B.parity_check_matrix()
    codim_C_A = len(parity_A.rows())
    codim_C_B = len(parity_B.rows())

    # Stage 1 - collecting the edges
    edges_A = set()
    edges_B = set()
    for g in G:
        for a in A:
            edges_A.add(AEdge(G, a, g))
        for b in B:
            edges_B.add(BEdge(G, g, b))

    assert len(edges_A) == len(list(G)) * len(A) / 2
    assert len(edges_B) == len(list(G)) * len(B) / 2

    # Stage 2 - iterating over edges and "injecting" constraints into squares
    for e in edges_A:
        a = e.a
        g = e.g
        for (j, b) in enumerate(B):  # B[j] = b
            square = Square(G, a, g, b)
            if square not in constraints:  # if no contraints were added on this square before
                constraints[square] = {}
            for (k, v) in enumerate(parity_A[:, j]):  # parity_A[k,j] = v
                constraints[square][row + k] = v[0]  # v is represted as a length 1 array
        row += codim_C_A

    for e in edges_B:
        g = e.g
        b = e.b
        for (i, a) in enumerate(A):  # A[i] = a
            square = Square(G, a, g, b)
            if square not in constraints:  # if no contraints were added on this square before
                constraints[square] = {}
            for (k, v) in enumerate(parity_B[:, i]):  # parity_B[k,i] = v
                constraints[square][row + k] = v[0]  # v is represted as a length 1 array
        row += codim_C_B

    assert len(constraints) == len(A) * len(B) * len(list(G)) / 4
    assert row == codim_C_A * len(edges_A) + codim_C_B * len(edges_B)

    return (constraints, row, edges_A, edges_B)

# Random generator sets <a name="sets"></a>

The following functions gets a group $G$, and two numbers, representing the size of two desired sets of generators. The number of elements of order 2 and of non order 2 are treated separatly (see the second and third argument to ``random_generators``).

The function ``get_AB_with_TNC`` returns two sets $A,B$ that satisfies the total no-conjugacy:
$$
\forall a\in A, b\in B, g\in G, \ g^{-1}ag\ne b
$$
It searches for these sets for in a brute-force way for several trials and exists if no such pair was found. 

In [5]:
def random_generators(G, n, n_order_2=0):
    """ Returns an inverse closed set of n + n_order_2 elements from G.


    Keyword arguments:
    G -- Sage group object.
    n -- number.
    n_order_2 -- number of elements of order 2. 
    
    Note:
    1) n has to be smaller than the number of elements in G.
    """

    list_G = list(G)
    assert n < len(list_G)
    gens = []
    # non order 2 generators
    i = 0
    while i < n / 2:
        c = random.choice(list_G)
        if c not in gens and c * c != G.identity():
            gens.append(c)
            gens.append(c.inverse())
            i += 1
    # order 2 generators
    i = 0
    while i < n_order_2:
        c = random.choice(list_G)
        if c not in gens and c * c == G.identity():
            gens.append(c)
            i += 1    
    return gens

def get_AB_with_TNC(G, n, n_order_2 = 0, trials = 100):
    """ Returns two sets of a group G of size n, for which that TNC condition hold.
        If no sets were found after # of trials, it exits with error.  


    Keyword arguments:
    G -- Sage group object.
    n -- number.
    n_order_2 -- number. 
    trials -- number of trials to find A,B that uphold TNC. 
    
    Note:
    1) size has to be smaller than the number of elements in G.
    """
    
    for _ in range(trials):
        A = random_generators(G,n,n_order_2)
        B = random_generators(G,n,n_order_2)
        violations = 0
        for a in A:
            for b in B:
                for g in G:
                    if a * g == g * b:
                        violations += 1
                        break
        if violations == 0:
            return A, B
    exit(1)

def get_AB_from_LPS(G,p,q):
    """ Returns two sets of a group of size p + 1, according to the Lubotzky, Phillips and Sarnak construction of
    Ramanujan Cayley graphs. 


    Keyword arguments:
    G -- Sage group object.
    p -- number.
    q -- number. 
    
    Note:
    1) p,q = 1 mod 4 
    2) Legendre symbol (q,p) = -1
    """
    assert p % 4 == 1
    assert q % 4 == 1
    assert kronecker (q , p) == -1
    F = IntegerModRing(q)
    i = int(F(-1).square_root())
    odd = [x for x in range(p) if int(x) % 2 != 0 and int(x) < p/2]
    even = [x for x in range(-p,p) if int(x) % 2 == 0]
    solutions = []
    for a0 in odd:
        for a1 in even:
            for a2 in even:
                for a3 in even:
                    if (a0*a0 + a1*a1 + a2*a2 + a3*a3) == p:
                        solutions.append((a0,a1,a2,a3))
    generators = []
    for s in solutions:
        a0 = s[0]
        a1 = s[1]
        a2 = s[2]
        a3 = s[3]
        generators.append(G((matrix(F, 2,2,[[a0+i*a1, a2+i*a3],[-a2+i*a3,a0-i*a1]]))))

    A = []
    for g in generators:
        A.append(g)
        A.append(g.inverse())

    random.shuffle(generators)
    B = []
    for g in generators:
        B.append(g)
        B.append(g.inverse())
    return A,B

# Decoding c3LTC <a name="decoding"></a>

The decoding function ``decode_via_edges`` and ``decode_via_vertices`` decode a word $w$. These functions different from the decoding algorithm presented in the original paper, and are conceptually similar to tensor decoding algorithm and expander code decoding. 
Similarly to algorithms for decoding in expander codes, ``decode_via_vertices`` decodes a word by locally correcting the local views of ''unsatisfied'' vertices (i.e., vertices whose local view is not a leagal tensor codeword). Similarly to decoding in tensor codes, ``decode_via_edges`` decodes the local view of ''unsatisfied'' edges (i.e., edges whose local view is not a leagal codeword in $C_A$ or $C_B$, depending on their type), and alternate between decoding all such ``AEdges`` followed by decoding of all ``BEdges`` (akin to the alternation between rows and columns in tensor decoding). 

In [6]:
def decode_via_edges(c3ltc, noisy_word):
    squares_to_values = {}
    for (i, v) in enumerate(noisy_word):
        squares_to_values[c3ltc.index_to_square[i]] = v
    init = True
    iterating_set_A = None
    iterating_set_B = None
    past_words = []
    word_from_square_values = c3ltc.square_to_value_to_word(squares_to_values)
    while (init or (
            len(iterating_set_A) != 0 or len(iterating_set_B) != 0)) and word_from_square_values not in past_words:
        past_words.append(word_from_square_values)
        if init:
            init = False
            iterating_set_A = set(c3ltc.edges_A)
            iterating_set_B = set(c3ltc.edges_B)
        new_iterating_set_A = set([])
        new_iterating_set_B = set([])

        for k,e in enumerate(iterating_set_A):
            a = e.a
            g = e.g
            i = c3ltc.A.index(a)
            local_word = vector(c3ltc.base_field, [0] * len(c3ltc.B))
            for (j, b) in enumerate(c3ltc.B):
                square = Square(c3ltc.G,a, g, b)
                local_word[j] = squares_to_values[square]
            try:
                corrected_localy = c3ltc.C_B.decode_to_code(local_word)
            except:
                new_iterating_set_A.add(AEdge(c3ltc.G, a, g))
                continue
            for (j, b) in enumerate(c3ltc.B):
                square = Square(c3ltc.G,a, g, b)
                if squares_to_values[square] != corrected_localy[j]:
                    new_iterating_set_A.add(AEdge(c3ltc.G, a, g * b))
                    new_iterating_set_B.add(BEdge(c3ltc.G, g, b))
                    new_iterating_set_B.add(BEdge(c3ltc.G, a * g, b))
                squares_to_values[square] = corrected_localy[j]
            if e in new_iterating_set_A:
                new_iterating_set_A.remove(e)

        for k,e in enumerate(iterating_set_B):
            g = e.g
            b = e.b
            j = c3ltc.B.index(b)
            local_word = vector(c3ltc.base_field, [0] * len(c3ltc.A))
            for (i, a) in enumerate(c3ltc.A):
                square = Square(c3ltc.G,a, g, b)
                local_word[i] = squares_to_values[square]
            try:
                corrected_localy = c3ltc.C_A.decode_to_code(local_word)
            except:
                new_iterating_set_B.add(BEdge(c3ltc.G, g, b))
                continue
            for (i, a) in enumerate(c3ltc.A):
                square = Square(c3ltc.G,a, g, b)
                if squares_to_values[square] != corrected_localy[i]:
                    new_iterating_set_B.add(BEdge(c3ltc.G, a * g, b))
                    new_iterating_set_A.add(AEdge(c3ltc.G, a, g))
                    new_iterating_set_A.add(AEdge(c3ltc.G, a, g * b))
                squares_to_values[square] = corrected_localy[i]
            if e in new_iterating_set_B:
                new_iterating_set_B.remove(e)

        iterating_set_A = new_iterating_set_A
        iterating_set_B = new_iterating_set_B
        word_from_square_values = c3ltc.square_to_value_to_word(squares_to_values)
    return word_from_square_values

def decode_via_vertices(c3ltc, noisy_word):
    n_a = len(c3ltc.C_A.parity_check_matrix().columns())
    n_b = len(c3ltc.C_B.parity_check_matrix().columns())
    M = MatrixSpace(c3ltc.base_field, n_a, n_b)
    squares_to_values = {}
    for (i, v) in enumerate(noisy_word):
        squares_to_values[c3ltc.index_to_square[i]] = v
    init = True
    iterating_set = None
    past_words = []
    word_from_square_values = c3ltc.square_to_value_to_word(squares_to_values)

    while (init or len(iterating_set)) != 0 and word_from_square_values not in past_words:
        past_words.append(word_from_square_values)
        if init:
            init = False
            iterating_set = set(c3ltc.G)
        new_iterating_set = set([])
        for g in iterating_set:
            local_view = numpy.zeros((n_a, n_b))
            for i, a in enumerate(c3ltc.A):
                for j, b in enumerate(c3ltc.B):
                    square = Square(c3ltc.G,a, g, b)
                    local_view[i][j] = squares_to_values[square]
            try:
                corrected_local_view = tensor_decoding(M(local_view), c3ltc.C_A, c3ltc.C_B)
            except:
                new_iterating_set.append(g)
                continue
            for i, a in enumerate(c3ltc.A):
                for j, b in enumerate(c3ltc.B):
                    square = Square(c3ltc.G,a, g, b)
                    if corrected_local_view[i][j] != local_view[i][j]:
                        new_iterating_set.add(a * g)
                        new_iterating_set.add(g * b)
                        new_iterating_set.add(a * g * b)
                    squares_to_values[square] = corrected_local_view[i][j]
            if g in new_iterating_set:
                new_iterating_set.remove(g)
        iterating_set = new_iterating_set
        word_from_square_values = c3ltc.square_to_value_to_word(squares_to_values)

    return word_from_square_values

# c3LTC object <a name="c3ltc"></a>

The class ``c3LTC`` generates an object of the new code. It receives two codes, $C_A,C_B$ a group $G$ and two generator sets $A,B$. The code generates parity check and generator matrices using the function ``embedding_local_parity_constraints_on_squares``. It also uses the library [spasm](https://github.com/cbouilla/spasm). to perform linear algebra functions, for performance improvements. 

The function ``local_codeword_on_vertex`` gets as input a word $w$ (possibly noisy) and a vertex $v$. It returns $w|_{X(v)}$, the restriction of $w$ to the squares $v$ sees in his local tensor-word view. 

In [7]:
class c3LTC:

    def __init__(self, C_A, C_B, G, A, B):
        assert len(C_A.generator_matrix().columns()) == len(A)
        assert len(C_B.generator_matrix().columns()) == len(B)
        assert C_A.base_field().characteristic() == C_B.base_field().characteristic()

        (sparse_constraints, count, self.edges_A, self.edges_B) = embedding_local_parity_constraints_on_squares(C_A, C_B, G, A, B)

        # process sparse constraints
        parity_constraints = numpy.zeros((count, len(sparse_constraints)))
        for (i, l) in enumerate(sparse_constraints):
            for k in sparse_constraints[l]:
                parity_constraints[k][i] = sparse_constraints[l][k]

        # additional mappings

        self.parity_constraints = parity_constraints
        self.square_to_index = {}
        self.index_to_square = {}
        self.squares = list(sparse_constraints)
        self.vertex_to_squares = {}
        self.vertex_to_neighbours_A = {}
        self.vertex_to_neighbours_B = {}
        self.square_to_vertices = {}
        self.n_vertices = len(list(G))

        list_G = list(G)

        for (i, l) in enumerate(self.squares):
            self.square_to_index[l] = i
            self.index_to_square[i] = l
        
        
        for g in G:
            view = numpy.zeros((len(A), len(B)))
            for (i, a) in enumerate(A):
                for (j, b) in enumerate(B):
                    view[i][j] = self.squares.index(Square(G,a, g, b))
            self.vertex_to_squares[list_G.index(g)] = view

        for g in G:
            view = []
            for (i, a) in enumerate(A):
                view.append(list_G.index(a * g))
            self.vertex_to_neighbours_A[list_G.index(g)] = view
        
        for g in G:
            view = []
            for (j, b) in enumerate(B):
                view.append(list_G.index(g * b))
            self.vertex_to_neighbours_B[list_G.index(g)] = view
        
        for (i, s) in enumerate(self.squares):
            view = []
            a = s.a
            g = s.g
            b = s.b
            view.append(int(list_G.index(a * g)))
            view.append(int(list_G.index(g * b)))
            view.append(int(list_G.index(a * g * b)))
            view.append(int(list_G.index(g * b)))
            self.square_to_vertices[i] = view

        # properties of the code

        self.A = A
        self.B = B
        self.C_A = C_A
        self.C_B = C_B
        self.G = G
        self.base_field = C_A.base_field()
        gen, par = row_reduce_and_orthogonal(sparse_constraints, C_A.base_field().characteristic(), count,len(sparse_constraints))
        M = MatrixSpace(self.base_field, gen.shape[0],
                        gen.shape[1])
        self.generator_matrix = M(gen)
        M = MatrixSpace(self.base_field, par.shape[0],
                        par.shape[1])
        self.parity_check_matrix = M(par)
        self.length = numpy.array(self.generator_matrix).shape[1]
        self.dimension = numpy.array(self.generator_matrix).shape[0]

    def square_to_value_to_word(self, squares_to_values):
        corrected_word = vector(self.base_field, [0] * len(squares_to_values))
        for square in self.square_to_index:
            corrected_word[self.square_to_index[square]] = squares_to_values[square]
        return corrected_word

    def syndrome(self, c):
        return self.parity_check_matrix * c
    
    def decode_via_edges(self, noisy_word):
        return decode_via_edges(self, noisy_word)
    
    def decode_via_vertices(self, noisy_word):
        return decode_via_vertices(self, noisy_word)

    def local_codeword_on_vertex(self, vertex, word):
        square_view = self.vertex_to_squares[vertex]
        rows = len(square_view)
        cols = len(square_view[0])
        local_view_values = numpy.array([0] * rows
                                        * cols).reshape((rows, cols))
        for i in range(rows):
            for j in range(cols):
                local_view_values[i][j] = int(word[int(self.vertex_to_squares[vertex][i][j])])
        return local_view_values

    def __repr__(self):
        rep = 'c3LTC'
        return rep


# Concrete Example <a name="example"></a>

Below is a concrete example for a construction of the new code with the following parameters:

- $G = PSL_2(7)$. 
- $C_A,C_B = RS[4,6]$

In [8]:
G = PSL(2,7)
A, B = get_AB_with_TNC(G, 6)
C_A = ReedSolomonCode(GF(7), Integer(6), Integer(4))
C_B = ReedSolomonCode(GF(7), Integer(6), Integer(4))
c3ltc = c3LTC(C_A, C_B, G, A,B)

[*] Start row reduce from c


[IO] loading 1512 x 2016 SMS matrix modulo 7... 12.1k NNZ [0.0s]
[CSR] Compressing... 12096 actual NZ, Mem usage = 102.8kbyte [0.00s]
LU : 1368 / 2016 [|L| = 0 / |U| = 189778] -- current density= (0.560 vs 0.112) --- rank >= 1343
[LU] testing for early abort...SUCCESS



LU : 167 / 168 [|L| = 0 / |U| = 223470] -- current density= (0.999 vs 0.890) --- rank >= 167


[*] Actual time in c 0.43617796897888184
[*] Finished row reduce from c


In [9]:
g = net.Network(notebook=True)
g.toggle_physics(False)
g.from_nx(show_graph(c3ltc))
g.show('graph.html')

Vertices that participate in square no. 1.

In [10]:
show_square(c3ltc,1)

,g,ag,gb,agb
1,96,122,59,122


Squares touching $v$. 

The numbers in the blue column are the $A$-neighbors of $v$.
The numbers in the red row are the $B$-neighbors of $v$.
Within the table, the entries correspond to the squares. 

In [11]:
local_view(c3ltc,1)

,34,133,26,63,10,24
9,359,656,168,536,146,657
25,31,466,144,258,467,468
61,1343,188,963,1498,250,1419
40,1122,1507,1093,264,708,1508
39,1461,1224,1204,1253,842,1462
60,365,783,784,785,786,787


Shows the squares of the local view of vertices $v_1,v_2$ (the common row is highlighted). 

The generating sets $A,B$ of the Cayley graph is ordered like: $(a_1,a_1^{-1}, a_2,a_2^{-1}...)$. That is, the generators are ordered in pairs of generator followed by its inverse. (There are no $a_i = a_i^{-1}$). Similarly for the $b$'s.

Therefore, if $v_2 = a_1 \cdot  v_1$ then in the local view of $v_2$, the second neighbour is going to be $v_1$. 

In [12]:
show_common(c3ltc, 1, c3ltc.vertex_to_neighbours_A[1][0], "A")

,34,133,26,63,10,24
9,359,656,168,536,146,657
25,31,466,144,258,467,468
61,1343,188,963,1498,250,1419
40,1122,1507,1093,264,708,1508
39,1461,1224,1204,1253,842,1462
60,365,783,784,785,786,787
,50,109,2,23,26,0
25,890,1073,1449,548,170,1368
1,359,656,168,536,146,657
117,567,652,948,491,949,245


# Encoding, Decoding and Testing <a name="tests"></a>

Below we provide several functions for encoding, decoding, and testing the properties of the generated code, as well as some supllementry functions. 


In [13]:
def word_with_k_injected_local_views(c3ltc, k):
    """ Create a random word with k local views "injected" with random tensor codewords. 
        Note that some vertices may not have a tensor codeword in their local view because the 
        function writes over pre-existing values. 

    Keyword arguments:
    c3ltc -- c3LTC object.
    k -- number. 
    """
    word = vector(c3ltc.base_field, [0] * c3ltc.length)
    for _ in range(k):
        v = random.randint(0,len(c3ltc.G)-1)
        local_view_flat = c3ltc.vertex_to_squares[v].reshape((C_A.length() * C_B.length()))
        random_local_word = random_tensor_word(c3ltc.C_A, c3ltc.C_B)
        for (i,b) in enumerate(random_local_word):
            word[int(local_view_flat[i])] = Integer(b)
    return word

def is_word_in_tensor_code(C_A,C_B, word):
    """ Checks if a matrix tensor word is in code by verifying that each row and column are 
        in the associated code. 

    Keyword arguments:
    C_A -- Sage code object.
    C_B -- Sage code object.
    word -- number. 
    """
    rows = word.shape[0]
    cols = word.shape[1]
    field = C_A.base_field()
    for i in range(rows):
        row = word[i,:]
        if 0 != numpy.count_nonzero(C_A.syndrome(vector(field, row))):
            return false
    for i in range(cols):
        col = word[:,i]
        if 0 != numpy.count_nonzero(C_B.syndrome(vector(field, col))):
            return false
    return true

def unsatisfied_vertices(c3ltc, word):
    """ Returns a list of vertices for which the local view in the given word is not in the
        tensor code. 

    Keyword arguments:
    c3ltc -- c3LTC object.
    word -- number. 
    """
    n_vertices = len(c3ltc.G)
    unsat = []
    for g in range(n_vertices):
        lv = c3ltc.local_codeword_on_vertex(g, word)
        if not is_word_in_tensor_code(c3ltc.C_A,c3ltc.C_B, lv):
            unsat.append(g)
    return unsat


def estimate_ltc_constant_with_injected_local_views(c3ltc, trials = 10, max_injected = 50):
    """ Returns an estimation of the local testability parameter by sampling a random noisy word and counting the 
        relative fraction of unsatisfied vertices. 
        The noise is generated by injecting random tensor words into the local view of vertices. 

    Keyword arguments:
    c3ltc -- c3LTC object.
    trials -- number. 
    max_injected -- number. 
    """
    c = Infinity
    n_vertices = len(c3ltc.G)
    for _ in range(trials):
        word = word_with_k_injected_local_views(c3ltc, max_injected)
        syndrome_weight = float(
            len(unsatisfied_vertices(c3ltc,word)) / n_vertices
        )
        error_weight = float(numpy.count_nonzero(word)/ c3ltc.length)
        if error_weight != 0:
            c = min(c, float(syndrome_weight / error_weight))
    return c

def estimate_ltc_constant_with_random_local_views(c3ltc, trials = 10, max_injected = 50, sparsity = 2):
    """ Returns an estimation of the local testability parameter by sampling a random noisy word and counting the 
        relative fraction of unsatisfied vertices. 
        The noise is sampled randomly. 

    Keyword arguments:
    c3ltc -- c3LTC object.
    trials -- number. 
    max_injected -- number. 
    sparsity -- number. 
    """
    c = Infinity
    n_vertices = len(c3ltc.G)
    for _ in range(trials):
        noisy_word, word = random_noisy_word(c3ltc, sparsity)
        syndrome_weight = float(
            len(unsatisfied_vertices(c3ltc,noisy_word)) / n_vertices
        )
        error_weight = float(numpy.count_nonzero(word - noisy_word)/ c3ltc.length)
        if error_weight != 0:
            c = min(c, float(syndrome_weight / error_weight))
    return c

Functions for random samplings (used in the functions above).

In [14]:
def random_vector(size, max_val, sparsity = 2):
    """ Retruns a random vector of a specified size, each entry sampled in the range between 
        0 and max_val. Sparsity reflects that a non-zero probability is sampled.
        The probability of a non-zero element to appear in the array is 1/(max_val - 1) * (1/2) ^ sparsity.

    Keyword arguments:
    size -- number.
    max_val -- number.
    sparsity -- number. 
    """
    return [random.randint(0,max_val)*numpy.random.binomial(1, pow(0.5,sparsity), 1)[0] for _ in range(size)]

def random_tensor_word(C_A, C_B):
    """ Retruns a random tensor codeword. It is achieved by sampling a random vector, multiplying
        it by the tensor of the generators matrices of C_A, C_B.
        The word returned is a flattening of the "matrix" view of the tensor codeword into a
        single vector by concatenating the rows of the respective matrix.
        Sparsity parameter is defined in the random_vector function. 

    Keyword arguments:
    C_A -- Sage code object.
    C_B -- Sage code object.
    sparsity -- number. 
    """
    field_char = C_A.base_field().characteristic()
    tensor_dimension = C_A.dimension() * C_B.dimension()
    rv = random_vector(tensor_dimension,field_char,0)
    message = vector(C_A.base_field(), rv)
    return (C_A.generator_matrix().tensor_product(C_B.generator_matrix()).transpose() * message)

def random_noisy_word(c3ltc, sparsity = 2):
    """ Retruns a noisy c3ltc codeword by encoding a random vector. 
    Sparsity parameter is defined in the random_vector function. 

    Keyword arguments:
    c3ltc -- c3LTC object.
    sparsity -- number. 
    """
    field = c3ltc.base_field
    error = vector(field, random_vector(c3ltc.length, field.characteristic(),sparsity))
    word = vector(field, random_word(c3ltc)) 
    return vector(field, word + error), vector(field, word)

def random_word(c3ltc):
    """ Retruns a c3ltc codeword by encoding a random vector. 

    Keyword arguments:
    c3ltc -- c3LTC object.
    """
    field = c3ltc.base_field
    rv = random_vector(c3ltc.dimension,field.characteristic(),0)
    message = c3ltc.generator_matrix.transpose() * vector(field, rv)    
    return message


Below we test the decoding algorithms on a noisy codeword, generated by adding random noise a codeword. First, we apply decoding along the edges, then decoding along the vertices. After each run, we print whether the decoding was succesful. 

In [15]:
noisy_word, word = random_noisy_word(c3ltc,2)
corrected = c3ltc.decode_via_edges(noisy_word)
print("Decoded correctly by edges?", corrected == word)
corrected = c3ltc.decode_via_vertices(word)
print("Decoded correctly by vertices?", corrected == word)

Decoded correctly by edges? True
Decoded correctly by vertices? True


Below we show the local view of a vertex. Above, the local view presented had the names of the squares that touch the vertex. Here, given a concrete (possibly noisy) word, we present the local $\textit{values}$ assigned to the squares. Hovering on a square value shows the number of that square. 

In [16]:
local_view_in_word(c3ltc,word,1)

,34,133,26,63,10,24
9,0,5,0,1,1,2
25,4,1,1,4,6,4
61,6,5,4,0,5,0
40,3,1,0,5,3,3
39,0,5,3,4,1,0
60,0,1,1,5,1,2


In [17]:
show_common_views_in_word(c3ltc,word,1,c3ltc.vertex_to_neighbours_A[1][0],"A")

,34,133,26,63,10,24
9,0,5,0,1,1,2
25,4,1,1,4,6,4
61,6,5,4,0,5,0
40,3,1,0,5,3,3
39,0,5,3,4,1,0
60,0,1,1,5,1,2
,50,109,2,23,26,0
25,2,3,4,3,6,6
1,0,5,0,1,1,2
117,2,1,6,4,4,4


In [18]:
show_common_views_in_word(c3ltc,word,1,c3ltc.vertex_to_neighbours_A[1][0],"A")

,34,133,26,63,10,24
9,0,5,0,1,1,2
25,4,1,1,4,6,4
61,6,5,4,0,5,0
40,3,1,0,5,3,3
39,0,5,3,4,1,0
60,0,1,1,5,1,2
,50,109,2,23,26,0
25,2,3,4,3,6,6
1,0,5,0,1,1,2
117,2,1,6,4,4,4


Below is a visualization that displays the unsatisfied vertices (for which the local view is not a legal tensor codeword). 

In [19]:
noisy_word, word = random_noisy_word(c3ltc,5)
print("Distance of noisy word:", numpy.count_nonzero(noisy_word-word), "/",c3ltc.length, "("+str(numpy.count_nonzero(noisy_word-word) / c3ltc.length * 100) + "%)")
g = net.Network(notebook=True)
g.toggle_physics(False)
g.from_nx(show_graph(c3ltc, special_vertices_set = unsatisfied_vertices(c3ltc, noisy_word)))
g.show('graph.html')

Distance of noisy word: 28 / 1512 (1.8518518518518516%)


Below we print a heuristic estimation of the generated code. See the functions ``estimate_ltc_constant_with_injected_local_views`` and ``estimate_ltc_constant_with_random_local_views``. 

In [20]:
print("LTC constant, structured noise", estimate_ltc_constant_with_injected_local_views(c3ltc))
print("LTC constant, random noise", estimate_ltc_constant_with_random_local_views(c3ltc, sparsity = 0))

LTC constant, structured noise 1.5077081192189106
LTC constant, random noise 1.2956298200514138
